<div style='text-align: right; font-size: 16px;'>
<br/>
<img src="images/HWR_logo.svg" alt="drawing" width="30%"/>
<br/>
Fachbereich Duales Studium Wirtschaft • Technik<br/>
ET1031 Mathematische Grundlagen III (Signale und Systeme)<br/>
Prof. Dr. Luis Fernando Ferreira Furtado, Berlin, 23.10.2025<br/>
</div>

# Kapitel 10. Nyquist-Shannon-Abtasttheorem

<hr style="border:solid #000000 1px;height:1px;">

In [ ]:
import numpy as np
import math
from si_prefix import si_format
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from ipywidgets import interact, FloatSlider, IntSlider, ToggleButtons, Select
from IPython.display import Audio, display, Latex, HTML, clear_output
import plotly.graph_objects as go
import plotly.express as px
import serial
import librosa
import warnings
warnings.filterwarnings("ignore")

def calc_freq_out(freq_s, freq_in):
    
    return abs(round(freq_in/freq_s,0)*freq_s - freq_in)

def plot_aliasing(freq_s, freq_in, phase_in):

    #### Time domain
    # Time axis
    f = freq_s
    N = 10
    t = np.arange(0, N*(1/f), 1/(f*100))
    n = np.arange(0, N + 1) / f # Normalized n-axis according to the sampling frequency

    # Calculate the frequency of the output signal
    freq_out = round(calc_freq_out(freq_s, freq_in), 1)
    if (freq_in == freq_s/2) and (phase_in in [90, 270]):
        freq_out=0
        
    # Display the values of the frequencies
    eq = r'\\f_{s}='+si_format(freq_s, precision=1)+'Hz'
    eq+= r'\quad f_{in}='+si_format(freq_in, precision=1)+'Hz'
    eq+= r'\quad f_{out}='+si_format(freq_out, precision=1)+'Hz'
    display(Latex(r'\begin{align}' + eq +'\end{align}'))  
    
    # Generate the signals according to their frequencies
    xt_in = np.cos(2*np.pi * freq_in * t + np.deg2rad(phase_in))   # x(t) for input
    xt_out = np.cos(2*np.pi * freq_out * t + np.deg2rad(phase_in)) # x(t) for output
    xn_out = np.cos(2*np.pi * freq_out * n + np.deg2rad(phase_in))    # x[n] for output

    # Plot Lollipop 
    def offset_signal(signal, marker_offset):
        if abs(signal) <= marker_offset:
            return 0
        return signal - marker_offset if signal > 0 else signal + marker_offset
    shapes=[]
    for i in range(len(n)):
        shape = dict(type='line', xref='x', yref='y', x0=n[i], y0=0, x1=n[i], y1=offset_signal(xn_out[i], 0.04), line=dict(color='#0000ff', width=1))
        shapes.append(shape)
    
    # Plot in time domain
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=n, y=xn_out, mode='markers', marker=dict(color='#0000ff'), name='x_out[n]'))
    fig.add_trace(go.Scatter(x=t, y=xt_out, mode='lines', line=dict(color='#0000ff'), name='x_out(t)'))    
    fig.add_trace(go.Scatter(x=t, y=xt_in, mode='lines', line=dict(color='#ff0000', dash='dot'), name='x_in(t)'))
    fig.update_layout(height=300, template='plotly_white', xaxis = dict(title_text='time [s]'), margin=dict(l=20, r=20, t=20, b=20),
                      legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),shapes=shapes)
    fig.show()    
   
    #### Frequency domain
    # Plot (frequency-domain)
    fig = go.Figure()
    # f_sampling
    fig.add_vrect(x0=0, x1=freq_s/2, fillcolor='#00B050', opacity=0.25, layer='below', line_width=0)    
    fig.add_trace(go.Scatter(x=[freq_s], y=[1], mode='markers', marker=dict(color='#00B050', size=10), name='f_s'))
    fig.add_vline(x=freq_s, line_color='#00B050', line_width=2)
    # f_in
    fig.add_trace(go.Scatter(x=[freq_in], y=[.95], mode='markers', marker=dict(color='#FF0000', size=10), name='f_in'))
    fig.add_vline(x=freq_in, line_color='#FF0000', line_width=2)
    # f_out
    fig.add_trace(go.Scatter(x=[freq_out], y=[.9], mode='markers', marker=dict(color='#0000FF', size=10), name='f_out'))
    fig.add_vline(x=freq_out, line_color='#0000FF', line_width=2)    

    fig.update_layout(height=250, template='plotly_white', margin=dict(l=20, r=20, t=20, b=20),
                     yaxis=dict(range=[0, 1.1], visible=False), xaxis=dict(title_text='frequency [Hz]', range=[0, int(2*max(freq_s,freq_in))]))
    fig.show()    

def plot_spectrogram(filename, duration, time=True, mel=True, n_fft=1024, filter={'type':'', 'freq':[]}):
        
    # Load the file in the time domain
    y, sr = librosa.load(filename, duration=duration)
    
    print(filter['type'])
    # Filter the signal    
    if filter['type'] != '':
        try:
            sos = signal.butter(filter['order'], filter['freq'], filter['type'], fs=sr*2, output='sos')
            y = signal.sosfilt(sos, y)
        except:
            pass
    
    # Transform from the time domain to the frequency domain 
    # Short-Time Fourier Transform (STFT) = FFT
    Y = librosa.stft(y, n_fft=n_fft)   
    
    # Trasform the power to DB
    Y = librosa.power_to_db(abs(Y) ** 2, ref=np.max)    
    
    # Transform from the time domain to the frequency domain (Mel-Scale)
    # FFT adatpted to the Mel-Scale
    if mel:
        Ymel = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft)
        Ymel = librosa.power_to_db(abs(Ymel) ** 2, ref=np.max) 
  
    # Plot signal
    fig = plt.figure(figsize=(14, 3*(time + 1 + mel)), facecolor='white')
    gs = GridSpec(nrows=(time + 1 + mel), ncols=1)    
    n = 0
    
    if time:
        ax = fig.add_subplot(gs[n, :])
        ax.set_title(label='Zeitbereich', loc='left')
        ax.set(xlim=[0, duration + .001])
        librosa.display.waveshow(y, sr=sr, ax=ax)
        n += 1
        
    ax = fig.add_subplot(gs[n, :])
    ax.set(xlim=[0, duration + .001])
    ax.set_title(label='Frequenzbereich', loc='left')
    librosa.display.specshow(Y, sr=sr*2, x_axis='time', y_axis='linear', ax=ax)
    n += 1
    
    if mel:
        ax = fig.add_subplot(gs[n, :])
        ax.set_title(label='Frequenzbereich - Mel-Skala', loc='left')
        ax.set(xlim=[0, duration + .001])
        librosa.display.specshow(Ymel, sr=sr, x_axis='time', y_axis='mel', ax=ax)
    
    plt.show()
    
    if filter['type'] != '':
        filter_text = ' | Filtertyp:[' + filter['type'] + '], Filterfreq.:' + str(filter['freq']) + 'Hz' +\
                      ' , Filterordnung:[' + str(filter['order']) + '.]'
    else:
        filter_text = ' | Ohne Filter'
        
    print(filename + filter_text)
    # Return y and sr for the audio play-button
    return y, sr

<hr style="border:solid #000000 1px;height:1px;">

#### Beispiel 10.1
<br/>
Berechnen Sie die Auflösung in µV eines Audiokanals eines ADC mit 16 Bits (kompatibel mit CD) unter Berücksichtigung einer Referenzspannung von 5 V.

In [ ]:
N = 16 		# number of bits
L = 2 ** N 	# quantization levels
U_ref = 5	# Volts 
resolution = U_ref / (L-1)
print('Quantisierungsauflösung:', resolution)

<hr style="border:solid #000000 1px;height:1px;">

#### Beispiel 10.2
<br/>
Berechnen Sie die Abtastperiode eines Audiokanals eines ADC (kompatibel mit CD) mit einer Abtastfrequenz von 44100 Hz.


In [ ]:
f_s = 44100
T_s = 1/f_s
print(T_s)

<hr style="border:solid #000000 1px;height:1px;">

#### Nyquist-Shannon-Abtasttheorem

In [ ]:
def update(freq_s, freq_in, phase_in):

    # Round the frequencies to avoid float comparison error
    freq_s = round(freq_s, 1)
    freq_in = round(freq_in, 1)
    
    # Call the function to plot the aliasing-effect
    plot_aliasing(freq_s, freq_in, phase_in)

freq_s = FloatSlider(value=400, min=1, max=1200, step=0.1, description='f_s [Hz]')
freq_in = FloatSlider(value=200, min=1, max=1200, step=0.1, description='f_in [Hz]')
phase_in = IntSlider(value=0, min=0, max=360, step=30, description='φ_in [°]')

interact(update, freq_s=freq_s, freq_in=freq_in, phase_in=phase_in);

<hr style="border:solid #000000 1px;height:1px;">

#### Beispiel 10.3
<br/>
Welches sind die gemessenen Ausgangsfrequenzen $f_out$ für die folgenden Eingangsfrequenzen $f_in$, wenn ein ADC-Umsetzer mit einer Abtastrate $f_s$ von 10 kHz verwendet wird?

In [ ]:
f_s = 10E3
f_in_a = 3E3
f_in_b = 5E3
f_in_c = 7E3
f_in_d = 11E3
f_in_e = 20E3
f_in_f = 99E3

print('a) Für f_s=10kHz und f_in=3kHz,  \t dann f_out =', si_format(calc_freq_out(f_s, f_in_a), precision=1) + 'Hz')
print('b) Für f_s=10kHz und f_in=11kHz, \t dann f_out =', si_format(calc_freq_out(f_s, f_in_b), precision=1) + 'Hz')
print('c) Für f_s=10kHz und f_in=36kHz, \t dann f_out =', si_format(calc_freq_out(f_s, f_in_c), precision=1) + 'Hz')
print('d) Für f_s=10kHz und f_in=122kHz,\t dann f_out =', si_format(calc_freq_out(f_s, f_in_d), precision=1) + 'Hz')
print()

def update(button):

    if button == '3 kHz':
        freq_in = f_in_a
    elif button == '5 kHz':
        freq_in = f_in_b
    elif button == '7 kHz':
        freq_in = f_in_c
    elif button == '11 kHz':
        freq_in = f_in_d
    elif button == '20 kHz':
        freq_in = f_in_e
    elif button == '99 kHz':
        freq_in = f_in_f               
    else:
        freq_in = 0
    
    # Call the function to plot the aliasing-effect
    plot_aliasing(f_s, freq_in, 0)

button = ToggleButtons(options=['3 kHz', '5 kHz', '7 kHz', '11 kHz', '20 kHz', '99 kHz'], description='Plot-Eingangsfrequenzen:')

interact(update, button=button);

<hr style="border:solid #000000 1px;height:1px;">

#### Praktische Anwendungen
__Arduino als Spektrumanalysator__
<br/>
Arduino liest das Mikrofon, berechnet die Fast Fourier Transformation und sendet einen Vektor mit den Amplituden für jedes Frequenzband über die serielle Schnittstelle.<br/><br/>
__Anmerkung:__ Python funktioniert problemlos mit der seriellen Kommunikation, allerdings hat Jupyter nicht viel Flexibilität, um die Bilder basierend auf den seriellen Daten zu aktualisieren.

In [ ]:
%matplotlib inline

# Set the serial port to communicate with the arduino
arduino = serial.Serial('COM3', 115200, timeout=.1)

# Set the number of reading cycles that should be executed in the jupyter notebook
cycles = 100

# Run for a configured ammount of time
while cycles >= 0:
    
    # Read the serial message and store it in the variable msg
    msg = arduino.readline()[:-3] # [:-3] gets rid of the new-line chars
    
    # Check if the message exists
    if msg:
        
        # Check if the message is valid"
        msg = str(msg).split(' ')
        if msg[0]=="b'START" and msg[-1]=="END'":
            
            # Cut the b'START" and the "END'"
            msg = msg[1:-1]
            
            # Slice the relevant variables and set the arrays to be plotted (X_f and f)
            freq_s = round(float(msg[0]),1)
            f_peak = round(float(msg[1]), 1)
            X_f = [float(f) for f in msg[2:]]
            f = np.arange(0, freq_s/2, freq_s/(2*len(X_f)))
            
            # Plot dynamicaly
            clear_output(wait=True)
            fig = plt.figure(figsize=(12, 4), facecolor='white')
            plt.bar(f, X_f, width=3)
            plt.title('[' + str(cycles) + ']   ' + 'Dominante Frequenz = ' + str(f_peak) + ' Hz')
            plt.ylabel('Amplitude'); plt.xlabel('Hz');
            plt.grid(color='black', alpha=0.25, linewidth=.5)
            plt.show()

            cycles -= 1
        
# Close the serial communication with the Arduino
arduino.close()

In [ ]:
arduino.close()